In [2]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [3]:
temperature = 0.0

class BusinessName(BaseModel):
    name: str = Field(description="The name of the business")
    rating_score: float = Field(description="The rating score of the business. 0 is the worst, 10 is the best.")

class BusinessNames(BaseModel):
    names: List[BusinessName] = Field(description="A list of business names")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=BusinessNames)

principles = """
- The name must be easy to remember.
- Use the {industry} industry and Company context to create an effective name.
- The name must be easy to pronounce.
- You must only return the name without any other text or characters.
- Avoid returning full stops, \n or any other characters.
- The maximum length of the name must be 10 characters.
"""

In [4]:
# Chat Model Output Parser:
chat = ChatOpenAI() 
template = """Generate 5 business names for a new start up company in the {industry} industry. 
You must follow the following principles: {principles}
{format_instructions}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

chat_prompt.output_parser = PydanticOutputParser(pydantic_object=BusinessNames)

# Get a chat completion from the formatted messages:
chat_response_output = chat(
    chat_prompt.format_prompt(
        principles=principles,
        industry="Data Science",
        format_instructions=parser.get_format_instructions(),
    ).to_messages()
)

pydantic_schema = parser.parse(chat_response_output.content)
print(pydantic_schema, "\n")
print(pydantic_schema.model_dump())

names=[BusinessName(name='DataTech', rating_score=8.5), BusinessName(name='InfoSci', rating_score=9.2), BusinessName(name='DataWave', rating_score=7.9), BusinessName(name='AnalytiX', rating_score=9.6), BusinessName(name='DataSense', rating_score=8.7)] 

{'names': [{'name': 'DataTech', 'rating_score': 8.5}, {'name': 'InfoSci', 'rating_score': 9.2}, {'name': 'DataWave', 'rating_score': 7.9}, {'name': 'AnalytiX', 'rating_score': 9.6}, {'name': 'DataSense', 'rating_score': 8.7}]}


In [19]:
# Alternatively you can write it like this:
chain = chat_prompt | chat | PydanticOutputParser(pydantic_object=BusinessNames)

chain.invoke({
    "principles": principles,
    "industry": "Data Science",
    "format_instructions": parser.get_format_instructions(),
})